In [71]:
# The maximum amount of PORTFOLIO_VAL to risk on a trade
MAX_RISK = 0.02

# How far to dip below support before triggering stoploss
SUPPORT_MARGIN = 0.01

# Target profit potential ratio relative to risk
TARGET_RATIO = 2

# Don't put more than this fraction of the portfolio in one stock
MAX_INVESTMENT = 0.8

def shares_to_buy(market: float, support: float) -> dict:
    """Calculate how many shares of a stock to buy given price"""
    result = {'market': market}
    result['stop'] = support * (1-SUPPORT_MARGIN)
    
    # Calculate target buy equity
    stock_risk = 1 - result['stop'] / market
    target = MAX_RISK * PORTFOLIO_VAL / stock_risk
    target = min(target, MAX_INVESTMENT * PORTFOLIO_VAL)
    target = min(target, LIQUIDITY - market / 2)
    
    # We cant invest less than one share
    if target < market:
        return None
    
    # Round target buy equity to a multiple of the market price
    result['shares'] = round(target / market)
    result['equity'] = market * result['shares']
    
    #  Recalculate based on rounding
    result['risk_percent'] = result['equity'] / PORTFOLIO_VAL * (1- result['stop'] / result['market'])
    result['risk'] = result['equity'] * (1 - result['stop'] / result['market'])
    result['target'] = (1 +result['risk'] * 2 / result['equity']) * market
    result['target_percent'] = result['target'] / market - 1
    return result

def print_result(data: dict):
    if not data:
        print("Stock is not a good buy")
        return
    
    REGEX = "Buy %i shares at $%0.2f worth $%0.2f with stop at $%0.2f"
    REGEX2 = "Risking $%0.2f for %0.2f percent of a $%0.2f portfolio"
    print(REGEX % (data['shares'], data['market'], data['equity'], data['stop']))
    print(REGEX2 % (data['risk'], data['risk_percent']*100, PORTFOLIO_VAL) )
    print("2:1 share price target = $%0.2f +%0.2f" % (data['target'], data['target_percent'] * 100))
    print("1.05x -> $%0.2f" % (data['equity'] * 0.05))
    print("1.10x -> $%0.2f" % (data['equity'] * 0.10))
    

In [90]:
# Value of the portfolio, used to calculate total risk
PORTFOLIO_VAL = 20000
LIQUIDITY = 20000

result = shares_to_buy(23.20, 22.67)
print_result(result)

Buy 529 shares at $23.20 worth $12272.80 with stop at $22.44
Risking $400.29 for 2.00 percent of a $20000.00 portfolio
2:1 share price target = $24.71 +6.52
1.05x -> $613.64
1.10x -> $1227.28
